# Machine Learning for Finance Freestyle

In this lab you'll be given the opportunity to apply everything you have learned to build a trading strategy for SP500 stocks. First, let's introduce the dataset you'll be using.

## The Data

Use BigQuery's magic function to pull data as follows:

    Dataset Name: ml4f
    Table Name: percent_change_sp500

The following query will pull 10 rows of data from the table:

In [1]:
%%bigquery df
SELECT 
    *
FROM
    `cloud-training-prod-bucket.ml4f.percent_change_sp500`
LIMIT
    10

In [2]:
df.head()

,symbol,Date,Open,Close,tomorrow_close,tomo_close_m_close,close_MIN_prior_5_days,close_MIN_prior_20_days,close_MIN_prior_260_days,close_MAX_prior_5_days,...,close_STDDEV_prior_20_days,close_STDDEV_prior_260_days,close_values_prior_260,days_on_market,scaled_change,s_p_scaled_change,normalized_change,company,industry,direction
0,BEAM,2008-03-12,65.27,65.31,64.88,-0.43,0.972439,0.972439,0.972439,1.000766,...,0.018391,0.090670,"[80.51, 79.67, 79.28, 80.32, 79.64, 79.87, 80....",7866,-0.006584,0.005127,-0.011711,Beam Inc,Consumer Staples,DOWN
1,ABT,1993-08-13,23.62,24.00,24.87,0.87,0.958333,0.958333,0.958333,1.025833,...,0.033533,0.095144,"[30.62, 30.62, 30.62, 30.5, 31.12, 30.62, 30.3...",2620,0.036250,0.004976,0.031274,Abbott Laboratories,Health Care,UP
2,BAX,2012-10-01,60.42,60.09,60.47,0.38,0.995673,0.966550,0.792977,1.017640,...,0.015421,0.058748,"[56.43, 54.56, 52.91, 53.47, 54.33, 55.87, 55....",7783,0.006324,0.000872,0.005452,Baxter Intl Inc,Health Care,STAY
3,BIG,2005-08-15,12.65,12.79,11.97,-0.82,0.964816,0.908522,0.788116,0.994527,...,0.029383,0.062441,"[11.67, 11.41, 11.6, 11.88, 11.94, 11.73, 11.4...",4476,-0.064113,-0.011776,-0.052337,Big Lots Inc,Consumer Discretionary,DOWN
4,BBY,1990-10-10,5.25,5.38,5.50,0.12,0.975836,0.975836,0.975836,1.092937,...,0.152138,0.357552,"[5.75, 5.88, 6.88, 7.88, 8.25, 8.13, 8.0, 7.88...",1386,0.022305,-0.016412,0.038717,Best Buy Co Inc,Consumer Discretionary,UP


As you can see, the table contains daily open and close data for SP500 stocks. The table also contains some features that have been generated for you using [navigation functions](https://cloud.google.com/bigquery/docs/reference/standard-sql/navigation_functions) and [analytic functions](https://cloud.google.com/bigquery/docs/reference/standard-sql/analytic-function-concepts). Let's dig into the schema a bit more. 

In [3]:
%%bigquery 
SELECT
    * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM
    `cloud-training-prod-bucket.ml4f`.INFORMATION_SCHEMA.COLUMNS
WHERE
    table_name = "percent_change_sp500"

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_hidden,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,cloud-training-prod-bucket,ml4f,percent_change_sp500,symbol,1,YES,STRING,NO,NO,NO,NaN
1,cloud-training-prod-bucket,ml4f,percent_change_sp500,Date,2,YES,DATE,NO,NO,NO,NaN
2,cloud-training-prod-bucket,ml4f,percent_change_sp500,Open,3,YES,FLOAT64,NO,NO,NO,NaN
3,cloud-training-prod-bucket,ml4f,percent_change_sp500,Close,4,YES,FLOAT64,NO,NO,NO,NaN
4,cloud-training-prod-bucket,ml4f,percent_change_sp500,tomorrow_close,5,YES,FLOAT64,NO,NO,NO,NaN
5,cloud-training-prod-bucket,ml4f,percent_change_sp500,tomo_close_m_close,6,YES,FLOAT64,NO,NO,NO,NaN
6,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MIN_prior_5_days,7,YES,FLOAT64,NO,NO,NO,NaN
7,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MIN_prior_20_days,8,YES,FLOAT64,NO,NO,NO,NaN
8,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MIN_prior_260_days,9,YES,FLOAT64,NO,NO,NO,NaN
9,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MAX_prior_5_days,10,YES,FLOAT64,NO,NO,NO,NaN


Most of the features, like `open` and `close` are pretty straightforward. The features generated using analytic functions, such as `close_MIN_prior_5_days` are best described using an example. Let's take the 6 most recent rows of data for IBM and reproduce the `close_MIN_prior_5_days` column. 

In [4]:
%%bigquery
SELECT 
    *
FROM
    `cloud-training-prod-bucket.ml4f.percent_change_sp500`
WHERE
    symbol = 'IBM'
ORDER BY 
    Date DESC
LIMIT 6

,symbol,Date,Open,Close,tomorrow_close,tomo_close_m_close,close_MIN_prior_5_days,close_MIN_prior_20_days,close_MIN_prior_260_days,close_MAX_prior_5_days,...,close_STDDEV_prior_20_days,close_STDDEV_prior_260_days,close_values_prior_260,days_on_market,scaled_change,s_p_scaled_change,normalized_change,company,industry,direction
0,IBM,2013-02-01,204.65,205.18,NaN,NaN,0.989716,0.937323,0.879813,0.998977,...,0.025808,0.031267,"[180.52, 188.52, 189.98, 191.93, 191.73, 190.9...",12860,NaN,-0.011539,NaN,Intl Business Machines Corp,Information Technology,STAY
1,IBM,2013-01-31,203.32,203.07,205.18,2.11,1.002216,0.947063,0.888955,1.009356,...,0.025220,0.031925,"[181.07, 180.52, 188.52, 189.98, 191.93, 191.7...",12859,0.010391,0.010053,0.000338,Intl Business Machines Corp,Information Technology,STAY
2,IBM,2013-01-30,203.69,203.52,203.07,-0.45,1.001867,0.941185,0.884434,1.007125,...,0.024643,0.032221,"[180.0, 181.07, 180.52, 188.52, 189.98, 191.93...",12858,-0.002211,-0.002563,0.000352,Intl Business Machines Corp,Information Technology,STAY
3,IBM,2013-01-29,204.34,203.90,203.52,-0.38,0.961648,0.930996,0.878666,1.005248,...,0.023987,0.032552,"[179.16, 180.0, 181.07, 180.52, 188.52, 189.98...",12857,-0.001864,-0.003900,0.002036,Intl Business Machines Corp,Information Technology,STAY
4,IBM,2013-01-28,204.85,204.93,203.90,-1.03,0.948958,0.926316,0.874250,1.000195,...,0.021542,0.032677,"[180.55, 179.16, 180.0, 181.07, 180.52, 188.52...",12856,-0.005026,0.005106,-0.010132,Intl Business Machines Corp,Information Technology,DOWN
5,IBM,2013-01-25,204.45,204.97,204.93,-0.04,0.944772,0.926136,0.874079,0.998780,...,0.018510,0.032874,"[182.32, 180.55, 179.16, 180.0, 181.07, 180.52...",12855,-0.000195,-0.001850,0.001655,Intl Business Machines Corp,Information Technology,STAY


For `Date = 2013-02-01` how did we arrive at `close_MIN_prior_5_days = 0.989716`? The minimum close over the past five days was `203.07`. This is normalized by the current day's close of `205.18` to get `close_MIN_prior_5_days = 203.07 / 205.18 = 0.989716`. The other features utilizing analytic functions were generated in a similar way. Here are explanations for some of the other features:

* __scaled_change__: `tomo_close_m_close / close`
* __s_p_scaled_change__: This value is calculated the same way as `scaled_change` but for the S&P 500 index. 
* __normalized_change__: `scaled_change - s_p_scaled_change` The normalization using the S&P index fund helps ensure that the future price of a stock is not due to larger market effects. Normalization helps us isolate the factors contributing to the performance of a stock_market.
* __direction__: This is the target variable we're trying to predict. The logic for this variable is as follows: 

    ```sql
    CASE 
        WHEN normalized_change < -0.01 THEN 'DOWN'
        WHEN normalized_change > 0.01 THEN 'UP'
        ELSE 'STAY'
    END AS direction
    ```

## Create classification model for `direction`

In this example, your job is to create a classification model to predict the `direction` of each stock. Be creative! You can do this in any number of ways. For example, you can use BigQuery, Scikit-Learn, or AutoML. Feel free to add additional features, or use time series models.   



### Establish a Simple Benchmark

One way to assess the performance of a model is to compare it to a simple benchmark. We can do this by seeing what kind of accuracy we would get using the naive strategy of just predicting the majority class. Across the entire dataset, the majority class is 'STAY'. Using the following query we can see how this naive strategy would perform.

In [5]:
%%bigquery
WITH subset as (
    SELECT 
        Direction
    FROM
        `cloud-training-prod-bucket.ml4f.percent_change_sp500`
    WHERE
        tomorrow_close IS NOT NULL
)
SELECT 
    Direction,
    100.0 * COUNT(*) / (SELECT COUNT(*) FROM subset) as percentage
FROM
    subset
GROUP BY
    Direction

,Direction,percentage
0,STAY,53.766049
1,UP,23.240681
2,DOWN,22.993271


So, the naive strategy of just guessing the majority class would have accuracy of around 54% across the entire dataset. See if you can improve on this. 

### Train Your Own Model

In [6]:
%%bigquery 
CREATE OR REPLACE MODEL `ai4f.directionDNNClassifier`
OPTIONS
  (model_type='DNN_CLASSIFIER', input_label_cols=['direction']) AS
SELECT
  symbol,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  direction
FROM
  `cloud-training-prod-bucket.ml4f.percent_change_sp500`
WHERE
  tomorrow_close IS NOT NULL
  AND EXTRACT(year FROM Date) = 2013

""


In [7]:
%%bigquery dfEval

SELECT * FROM ML.EVALUATE(MODEL ai4f.directionDNNClassifier,
(
SELECT
  symbol,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  direction
FROM
  `cloud-training-prod-bucket.ml4f.percent_change_sp500`
WHERE
  tomorrow_close IS NOT NULL
  AND EXTRACT(year FROM Date) = 2012
))

In [9]:
dfEval

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.310904,0.299175,0.333,0.270956,1.153463,0.502827


In [10]:
%%bigquery dfPredict

SELECT * FROM ML.PREDICT(MODEL ai4f.directionDNNClassifier,
(
SELECT
  Date,
  symbol,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  direction AS ActualDirection
FROM
  `cloud-training-prod-bucket.ml4f.percent_change_sp500`
WHERE
  tomorrow_close IS NOT NULL
  AND EXTRACT(year FROM Date) = 2011
  AND EXTRACT(month FROM Date) = 12
  AND symbol = 'IBM'
))

In [11]:
dfPredict

,predicted_direction,predicted_direction_probs,Date,symbol,close_MIN_prior_5_days,close_MIN_prior_20_days,close_MAX_prior_5_days,close_MAX_prior_20_days,ActualDirection
0,STAY,"[{'label': 'UP', 'prob': 0.2722116708755493}, ...",2011-12-22,IBM,0.996869,0.972643,1.029884,1.068776,STAY
1,DOWN,"[{'label': 'UP', 'prob': 0.24072584509849548},...",2011-12-29,IBM,0.974702,0.974702,0.993393,1.045010,STAY
2,STAY,"[{'label': 'UP', 'prob': 0.30264806747436523},...",2011-12-15,IBM,1.006614,0.944421,1.037764,1.037764,DOWN
3,DOWN,"[{'label': 'UP', 'prob': 0.23078176379203796},...",2011-12-28,IBM,0.986304,0.983423,1.017664,1.057449,STAY
4,STAY,"[{'label': 'UP', 'prob': 0.29102766513824463},...",2011-12-20,IBM,0.976768,0.945631,1.020882,1.039094,DOWN
5,STAY,"[{'label': 'UP', 'prob': 0.3012154698371887}, ...",2011-12-19,IBM,1.003718,0.968123,1.050796,1.063809,STAY
6,STAY,"[{'label': 'UP', 'prob': 0.34357231855392456},...",2011-12-07,IBM,0.968822,0.912445,0.994280,0.994280,STAY
7,STAY,"[{'label': 'UP', 'prob': 0.3395346403121948}, ...",2011-12-06,IBM,0.937804,0.917695,0.989116,0.989116,STAY
8,STAY,"[{'label': 'UP', 'prob': 0.33943337202072144},...",2011-12-09,IBM,0.974815,0.910053,0.997379,0.997379,STAY
9,DOWN,"[{'label': 'UP', 'prob': 0.25525403022766113},...",2011-12-21,IBM,1.007825,0.975698,1.039952,1.072133,STAY
